# TF Eager Tutorial (Define by Run)

정리 및 요약 by Ryah Shin

[참고1: 구글 블로그](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html)

[참고2: 구글 깃허브](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/g3doc/guide.md)

[Eager 모드 설치방법(TF Nightly)](https://github.com/tensorflow/tensorflow#installation)

In [2]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()

## Gradients

In [5]:
def square(x):
    return tf.multiply(x, x)

grad = tfe.gradients_function(square)

print(square(3.))
print(grad(3.)) #x^2 -> 2x -> 6

#2차 gradients_function
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])

print(gradgrad(3.))

def abs(x):
    return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]

tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=59, shape=(), dtype=float32, numpy=6.0>]
[<tf.Tensor: id=75, shape=(), dtype=float32, numpy=2.0>]
[<tf.Tensor: id=84, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=94, shape=(), dtype=float32, numpy=-1.0>]


## Custom Grad

Custom Gradient 제작하기. 

주로 cross entropy나 log likelyhood에 쓰이는 예제로 log(1 + e^x) 제작

In [10]:
def log1pexp(x):
    return tf.log(1 + tf.exp(x))

grad_log1pexp = tfe.gradients_function(log1pexp)

print(grad_log1pexp(0.)) # [0.5]

print(grad_log1pexp(100.)) # x = 100, nan

[<tf.Tensor: id=115, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=126, shape=(), dtype=float32, numpy=nan>]


In [13]:
@tfe.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad
grad_log1pexp = tfe.gradients_function(log1pexp)

# Gradient at x = 0 works as before.
print(grad_log1pexp(0.))
# [0.5]
# And now gradient computation at x=100 works as well.
print(grad_log1pexp(100.))
# [1.0]

[<tf.Tensor: id=138, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=150, shape=(), dtype=float32, numpy=1.0>]


# Building Models

MNIST 2 Layer모델을 간단하게 Class로 만드는 예제

tfe.Network: 기본적으로 layer의 Container역할을 하여, 다른 NW객체에 임비디드 되어 NW객체가 된다.

추가로, inspection, saving, & restoring에 도움을 준다.

In [4]:
class MNISTModel(tfe.Network):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(units=10))
        self.layer2 = self.track_layer(tf.layers.Dense(units=10))
    def call(self, input):
        """모델 실행"""
        result = self.layer1(input)
        result = self.layer2(result)
        return result
    
#placeholder나 session에 대한 기능이 없고, input을 pass되면 자동으로 세팅 됨

In [5]:
# 테스트 데이터셋 생성하기
model = MNISTModel()
batch = tf.zeros([1, 1, 784])
print(batch.shape)
result = model(batch)
print(result)

(1, 1, 784)
tf.Tensor([[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]], shape=(1, 1, 10), dtype=float32)


In [8]:
#학습을 위한 loss func, grad, 그리고 업데이트

#1. loss func
def loss_function(model, x, y):
    y_ = model(x)
    return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)

#2. training loop
#implicit_gradients(): 모든 TF 값에 대한 미분을 계산한다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate =0.001)
for (x, y) in tfe.Iterator(dataset):
    grads = tfe.implicit_gradients(loss_function)(model, x, y)
    optimizer.apply_gradients(grads)

AttributeError: 'EagerTensor' object has no attribute '_as_variant_tensor'

In [ ]:
#GPU 사용법
#optimizer.min을 통해서, 짧게 작성하였지만, apply_gradients()기능을 써도 가능

with tf.device("/gpu:0"):
    for (x, y) in tfe.Iterator(dataset):
        optimizer.minimize(lambda: loss_function(model, x, y))

## Using Ear with Graphs

- Eagar 자체는 개발하고 디버깅 할 때 좋은 기능을 가지지만, Tensorflow graph형식이 분산 학습, 성능 최적화, 상용개발에 더 적합
- 현재 모델을 graph형태로 변경하기 위해서는, eager를 disable하고 실행하면 됨
- 관련 예제 코드: [MNIST with Eager](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/mnist)
- 위의 예제 코드는 checkpoints를 저장하고 불러올 수 있기 때문에, 상용에도 적합하다.

## 현재 활용하고 있는 코드의 변화
 
- 현재 사용하고 있는 데이터에서, 형태를 tf.data로 변경하는것을 추천 드립니다.
  - [참고링크 1](https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html)
  - [참고링크 2](https://www.tensorflow.org/programmers_guide/datasets)
- tf.layer.Conv2D()와 같은 Object-oriented 기능 활용을 추천 (Explict storage for variables
- 대부분의 모델이 eagar로 활용이 가능하지만, dynamic 모델에 대한 control flow같은 경우는 추가로 검토가 필요
- tfe.enable_eagar_execution()을 활용하면, 끌수가 없기 때문에 Python 세션을 재시작 추천